<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/25_4_Lenta_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В домашней работе необходимо с помощью AutoKeras или KerasTuner найти оптимальную модель для решения одной из следующей задач:

1. На 3 балла. Обучите модель с точностью не менее 90% предсказывать сарказм в новостных заголовках. Составьте 5 произвольных заголовков, которых нет в датасете и проверьте на них обученную модель, сделайте выводы. Ссылка на [датасет](https://storage.yandexcloud.net/academy.ai/Sarcasm_Headlines_Dataset_v2.json.zip).
2. На 4 балла. Используйте [русский корпус новостей от Lenta.ru](https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta/data) подберите и обучите модель классифицировать новости по заголовкам на классы (поле topic в датасете). Используйте 9 самых часто встречаемых топиков и 10-й для остальных, не вошедших в 9 классов. Оцените модель с помощью отчета о классификации, сделайте выводы.  
3. На 5 баллов. Найдите публичный датасет по обращениям граждан в администрацию, техническую поддержку или за консультацией. Обучите модель классифицировать обращения по тематикам. Сформируйте отчет о классификации и матрицу ошибок.

In [58]:
!pip install autokeras==1.1.0 tensorflow==2.15.1 keras-nlp==0.5.1

In [63]:
import tensorflow as tf
import keras_nlp as nlp
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import autokeras as ak
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import kagglehub

In [64]:
path = kagglehub.dataset_download("yutkin/corpus-of-russian-news-articles-from-lenta")
df = pd.read_csv(path + '/lenta-ru-news.csv', )

<ipython-input-64-30f204d83ad7>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + '/lenta-ru-news.csv')


In [65]:
df.columns

Index(['url', 'title', 'text', 'topic', 'tags', 'date'], dtype='object')

In [66]:
df = df.drop('url', axis=1)
df = df.drop('tags', axis=1)
df = df.drop('date', axis=1)
df = df.drop('title', axis=1)

In [67]:
df.head(3)

,text,topic
0,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека
1,"Министерство народного просвещения, в виду про...",Библиотека
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека


In [68]:
df['topic'].value_counts()[:9]

,count
topic,
Россия,160445
Мир,136621
Экономика,79528
Спорт,64413
Культура,53797
Бывший СССР,53402
Наука и техника,53136
Интернет и СМИ,44663
Из жизни,27605


In [69]:
X_train, X_tmp, y_train, y_tmp = train_test_split(np.array(df.text), np.array(df.topic), test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5)

In [70]:
n = random.randint(0, len(df))
print('Форма входных данных: ', X_train.shape)
print('Форма выходных меток: ', y_train.shape)
print('Пример текста: ', X_train[n])

Форма входных данных:  (640780,)
Форма выходных меток:  (640780,)
Пример текста:  В сирийском городе Растан, куда для подавления волнений введены правительственные войска, погиб как минимум 41 человек. Об этом  Reuters в среду, 1 июня, сообщил один из сирийских правозащитников, подчеркнувший, что речь идет только о жертвах среди гражданского населения. Власти Сирии, в свою очередь, в среду сообщили о трех погибших в Растане военнослужащих. В заявлении, распространенном сирийским государственным информагентством, говорится, что военнослужащие погибли в боях  "с вооруженными  террористами, которые запугивают и убивают мирных граждан ." Войска в города Растан и Талбису, расположенные в центре страны, были введены в воскресенье, 29 мая. Перед вводом войск в обоих городах отключили электричество, водоснабжение, интернет и мобильную связь. Волнения в Сирии продолжаются уже более двух месяцев. Акции протеста начались в городе Дараа на юге страны, где для разгона демонстрантов были применены ж

In [71]:
del df, X_tmp, y_tmp

In [75]:
X_train = np.asarray(X_train).astype(np.float32)

# y_train=np.asarray(y_train).astype(np.float32)

ValueError: could not convert string to float: 'Крымский государственный океанариум, который готовит боевых дельфинов, перешел в подчинение Министерству обороны России. Об этом сообщает РИА Новости со ссылкой на исполняющего обязанности губернатора Севастополя Сергея Меняйло. Животных будут тренировать по новым программам для Военно-морского флота. Кроме того, по его словам, четыре дельфина из этого океанариума, которые ранее работали по программе детской терапии, переведены в центры дельфинотерапии. «Они помогают детям в лечении таких сложных болезней, как ДЦП, психические расстройства и психологические стрессы»,\xa0— отметил Меняйло. Подготовка боевых морских животных активно проводилась в Крыму при СССР, однако после распада Советского Союза была прервана и возобновилась только в 2012 году. До присоединения полуострова к России дельфинов тренировали по советским программам для Военно-морских сил Украины. Обученный дельфин умеет находить оружие и военную технику на дне моря. Животные

In [73]:
clf = ak.TextClassifier(overwrite=True, max_trials=2, objective='val_accuracy')

result_training = clf.fit(X_train, y_train, epochs=4, validation_data=(X_val, y_val))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).